In [1]:
import pandas as pd
df = pd.read_csv('tracks_features.csv')
len(df)

1204025

In [2]:
cols_to_drop = [
                "id",
                "album_id",
                "artist_ids",
                "track_number",
                "disc_number",
                "explicit",
                "year",
                "release_date"
]
df = df.drop(columns=cols_to_drop)
df.head(3)

,name,album,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Testify,The Battle Of Los Angeles,['Rage Against The Machine'],0.470,0.978,7,-5.399,1,0.0727,0.0261,0.000011,0.356,0.503,117.906,210133,4.0
1,Guerrilla Radio,The Battle Of Los Angeles,['Rage Against The Machine'],0.599,0.957,11,-5.764,1,0.1880,0.0129,0.000071,0.155,0.489,103.680,206200,4.0
2,Calm Like a Bomb,The Battle Of Los Angeles,['Rage Against The Machine'],0.315,0.970,7,-5.424,1,0.4830,0.0234,0.000002,0.122,0.370,149.749,298893,4.0


In [3]:
# Convert name, album, artists column to strings
df['name'] = df['name'].astype(str)
df['album'] = df['album'].astype(str)
df['artists'] = df['artists'].astype(str)

In [4]:
def isEnglish(s):
  return s.isascii()

print(isEnglish("Test"))
print(isEnglish("_1991_اف_جي2"))

True
False


In [5]:
song_mask = df['name'].apply(isEnglish)
len(df)

1204025

In [6]:
english_name_df = df[song_mask]
len(english_name_df)

1152511

In [7]:
album_mask = english_name_df['album'].apply(isEnglish)
english_name_and_album_df = english_name_df[album_mask]
len(english_name_and_album_df)

1126210

In [8]:
artists_mask = english_name_and_album_df['artists'].apply(isEnglish)
english_name_album_artists_df = english_name_and_album_df[artists_mask]
len(english_name_album_artists_df)

1079362

In [9]:
def any_spec_char(s):
  special_characters = "!@#$%:^&*-+?_[]1234567890=<>/\""
  return not any(c in special_characters for c in s)

def any_spec_album_char(s):
  special_characters = "!@#$%:^&*+?_=[]1234567890<>/\""
  return not any(c in special_characters for c in s)

def any_spec_artists_char(s):
  special_characters = "!@#$%:^&*+?\_=<>/"
  return not any(c in special_characters for c in s)

In [10]:
clean_name_mask = english_name_and_album_df['name'].apply(any_spec_char)
clean_name_df = english_name_and_album_df[clean_name_mask]
len(clean_name_df)

853885

In [11]:
clean_album_mask = clean_name_df['album'].apply(any_spec_album_char)
clean_name_and_album_df = clean_name_df[clean_album_mask]
len(clean_name_and_album_df)

699846

In [12]:
clean_artists_mask = clean_name_and_album_df['artists'].apply(any_spec_artists_char)
clean_name_album_artists_df = clean_name_and_album_df[clean_artists_mask]
len(clean_name_album_artists_df)

676215

Convert to Title Case

In [13]:
def convert_to_title_case(s):
  return s.title()

In [14]:
clean_name_album_artists_df['name'] = clean_name_album_artists_df['name'].apply(convert_to_title_case)
clean_name_album_artists_df['album'] = clean_name_album_artists_df['album'].apply(convert_to_title_case)
clean_name_album_artists_df['artists'] = clean_name_album_artists_df['artists'].apply(convert_to_title_case)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [17]:
clean_name_album_artists_df.head(1)

,name,album,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Testify,The Battle Of Los Angeles,['Rage Against The Machine'],0.47,0.978,7,-5.399,1,0.0727,0.0261,0.000011,0.356,0.503,117.906,210133,4.0


Check Profanity

In [18]:
pip install better_profanity

     |████████████████████████████████| 46 kB 1.9 MB/s 


In [19]:
from better_profanity import profanity

In [20]:
def check_profanity(s):
  return not profanity.contains_profanity(s)

In [ ]:
profanity_name_mask = clean_name_album_artists_df['name'].apply(check_profanity)

In [22]:
final_cleaned_df = clean_name_album_artists_df[profanity_name_mask]
len(final_cleaned_df)

665051

In [24]:
final_cleaned_df.head(3)

,name,album,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Testify,The Battle Of Los Angeles,['Rage Against The Machine'],0.470,0.978,7,-5.399,1,0.0727,0.0261,0.000011,0.356,0.503,117.906,210133,4.0
1,Guerrilla Radio,The Battle Of Los Angeles,['Rage Against The Machine'],0.599,0.957,11,-5.764,1,0.1880,0.0129,0.000071,0.155,0.489,103.680,206200,4.0
2,Calm Like A Bomb,The Battle Of Los Angeles,['Rage Against The Machine'],0.315,0.970,7,-5.424,1,0.4830,0.0234,0.000002,0.122,0.370,149.749,298893,4.0


In [25]:
from sklearn.utils import shuffle
final_cleaned_df = shuffle(final_cleaned_df)

In [26]:
final_cleaned_df.head(3)

,name,album,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
1085833,High Roller,Where Good Marbles Go To Die (Marble Madness),"['Stemage', 'Viking Guitar']",0.290,0.974,8,-7.252,1,0.1020,0.00016,0.735000,0.429,0.5340,180.020,294587,4.0
416994,Only The Lonely,With Light And With Love,['Woods'],0.562,0.883,9,-6.334,1,0.0518,0.00613,0.081600,0.304,0.6670,110.900,202061,4.0
894310,A Lot'S Gonna Change,Titanic Rising,['Weyes Blood'],0.222,0.342,1,-10.164,1,0.0308,0.63500,0.000808,0.124,0.0997,140.057,261947,4.0


In [28]:
final_cleaned_df.to_csv('spotify_songs_dataset.csv', index=False)